In [1]:
documents = ["bmad_doc/bmad-manual.pdf"]

In [7]:
%load_ext autoreload
%autoreload 1
import glob
import os
%aimport clean

tex_files = glob.glob("bmad_doc/*.tex")
for file in tex_files:
    filename_base = os.path.splitext(os.path.basename(file))[0]
    
    # Create the output file path
    output_file = os.path.join("clean_bmad_doc", f"{filename_base}.txt")
    
    # Process the file
    clean.clean_latex_file(file, output_file)

In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader

# Example: Load all PDFs from a directory
loader = DirectoryLoader(
    "clean_bmad_doc",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
documents = loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,  # Adjust based on your needs
    chunk_overlap=100,
    separators=[
        
        "\n\n",
        "\n",
        " ",
        ""
    ]
)
texts = text_splitter.split_documents(documents)

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Lightweight local model
)

In [4]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("faiss_clean")

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector_store = FAISS.load_local("faiss_clean", embeddings,allow_dangerous_deserialization=True)

In [7]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})  # Fetch top k results

In [8]:
# Test query
query = "quadrupole tracking methods"
results = retriever.invoke(query)

# Check retrieved text
for doc in results:
    print('\n!--------------------\n')
    print(doc.page_content)



!--------------------

chapter Tracking, Spin, and Transfer Matrix Calculation Methods

Bmad allows for a number of methods that can be use to "track" a particle through a lattice
element. Here "track" can mean one of three things:
  1) Calculate a particle's phase space coordinates at the exit 
     end of the element given the coordinates at the entrance end.
  2) Calculate the linear transfer map (Jacobian) through an element
     about a given reference orbit.
  3) Calculate the a particle's spin orientation at the exit end 
     of the element given the coordinates at the beginning.
The different tracking methods that are available have different advantages and disadvantages in
terms of speed, symplecticity, etc.  What tracking method is used, is selected on an
element--by--element basis using the attributes:
  tracking_method      = <Switch>   ! phase space tracking method.
  mat6_calc_method     = <Switch>   ! 6x6 transfer matrix calculation.
  spin_tracking_method = <Switch>  